In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import random
import nltk
import re
import string
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
en_stops = set(stopwords.words('english'))

# nltk.download('punkt')
nltk.download('stopwords')
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers --q

     |████████████████████████████████| 4.9 MB 4.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.8 MB/s 
     |████████████████████████████████| 120 kB 53.0 MB/s 


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

### data1

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/labeled_data.csv")
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
data = data.drop(['Unnamed: 0'], axis=1)

In [ ]:
data['tweet'][0]

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data['sentence'] = data['tweet'].apply(data_cleaner)

In [ ]:
data.head()

,count,hate_speech,offensive_language,neither,class,tweet,sentence
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt woman complain cleaning house amp man alway...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad cuffin dat hoe s...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt dawg rt sbaby life ever fuck bitch start cr...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt look like tranni
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt shit hear might true might faker bitch told ya


In [ ]:
def fix_text(text):
    text = text.replace('rt', '')
    # text = text.replace('   ', '')
    return text.strip()
data["sentence"] =  data["sentence"].apply(fix_text)

In [ ]:
filter = data["sentence"] != ""
final_data = data[filter]

In [ ]:
final_data.dropna(inplace = True)
final_data.isna().sum()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
sentence              0
dtype: int64

In [ ]:
final_data.head(3)

,count,hate_speech,offensive_language,neither,class,tweet,sentence
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,woman complain cleaning house amp man always t...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,boy dats cold tyga dwn bad cuffin dat hoe st plac
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,dawg sbaby life ever fuck bitch sta cry confu...


In [ ]:
final_data['intent'] = final_data['class'].map({0:'Hate',1:'Offensive',2:'Neither'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_data['intent'].value_counts()

Offensive    19181
Neither       4156
Hate          1425
Name: intent, dtype: int64

In [ ]:
data1 = final_data[['sentence','intent']]
data1

,sentence,intent
0,woman complain cleaning house amp man always t...,Neither
1,boy dats cold tyga dwn bad cuffin dat hoe st plac,Offensive
2,dawg sbaby life ever fuck bitch sta cry confu...,Offensive
3,look like tranni,Offensive
4,shit hear might true might faker bitch told ya,Offensive
...,...,...
24778,muthaf lie pearls right tl trash mine bible sc...,Offensive
24779,gone broke wrong hea baby drove redneck crazi,Neither
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive
24781,youu got wild bitches tellin li,Offensive


In [ ]:
data1.sentence.tolist()

['woman complain cleaning house amp man always take trash',
 'boy dats cold tyga dwn bad cuffin dat hoe st plac',
 'dawg  sbaby life ever fuck bitch sta cry confused shit',
 'look like tranni',
 'shit hear might true might faker bitch told ya',
 'shit blows claim faithful somebody still fucking ho',
 'sit hate another bitch got much shit go',
 'cause tired big bitches coming us skinny girl',
 'amp might get ya bitch back amp that',
 'hobbies include fighting mariam bitch',
 'keeks bitch curves everyone lol walked conversation like smh',
 'murda gang bitch gang land',
 'hoes smoke losers yea go ig',
 'bad bitches thing lik',
 'bitch get',
 'bitch nigga miss',
 'bitch plz whatev',
 'bitch lov',
 'bitches get cut everyday b',
 'black bottle amp bad bitch',
 'broke bitch cant tell noth',
 'cancel bitch like nino',
 'cant see hoes wont chang',
 'fuck bitch dont even suck dick kermit videos bout fuck ig',
 'got ya bitch tip toeing hardwood floor',
 'pussy lips like heaven door',
 'hoe hit',


In [ ]:
tokenize = final_data[['sentence','intent']]
tokenize

,sentence,intent
0,woman complain cleaning house amp man always t...,Neither
1,boy dats cold tyga dwn bad cuffin dat hoe st plac,Offensive
2,dawg sbaby life ever fuck bitch sta cry confu...,Offensive
3,look like tranni,Offensive
4,shit hear might true might faker bitch told ya,Offensive
...,...,...
24778,muthaf lie pearls right tl trash mine bible sc...,Offensive
24779,gone broke wrong hea baby drove redneck crazi,Neither
24780,young buck wanna eat dat nigguh like aint fuck...,Offensive
24781,youu got wild bitches tellin li,Offensive


In [ ]:
tokenize['class'].value_counts()

1    19181
2     4157
0     1425
Name: class, dtype: int64

In [ ]:
tokenize.to_csv(r'Dataset_1.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data1['input_ids']=data1['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])#do_not_pad
data1.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sentence,intent,input_ids
0,woman complain cleaning house amp man always t...,Neither,"[101, 2450, 17612, 9344, 2160, 23713, 2158, 24..."
1,boy dats cold tyga dwn bad cuffin dat hoe st plac,Offensive,"[101, 2879, 23755, 2015, 3147, 5939, 3654, 104..."
2,dawg sbaby life ever fuck bitch sta cry confu...,Offensive,"[101, 4830, 27767, 24829, 21275, 2166, 2412, 6..."


In [ ]:
print(data1['input_ids'][0])
print(tokenizer.decode(data1['input_ids'][0]))

[CLS] woman complain cleaning house amp man always take trash [SEP]


In [ ]:
data1['intent'] = data1['intent'].map({'Hate':0,'Offensive':1,'Neither':2})
data1.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sentence,intent,input_ids
0,woman complain cleaning house amp man always t...,2,"[101, 2450, 17612, 9344, 2160, 23713, 2158, 24..."
1,boy dats cold tyga dwn bad cuffin dat hoe st plac,1,"[101, 2879, 23755, 2015, 3147, 5939, 3654, 104..."


In [ ]:
tokenize_data1 = data1[['input_ids','intent']]
tokenize_data1.head(2)

,input_ids,intent
0,"[101, 2450, 17612, 9344, 2160, 23713, 2158, 24...",2
1,"[101, 2879, 23755, 2015, 3147, 5939, 3654, 104...",1


In [ ]:
tokenize_data1.to_csv(r'dataset1_tokenized.txt', header=True, index=True, sep=',', mode='a')

### data2

In [ ]:
data2 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_2.csv")
data2.head(3)

,Unnamed: 0,X_unit_id,X_golden,X_unit_state,X_trusted_judgments,X_last_judgment_at,does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech.confidence,X_created_at,orig__golden,...,orig__trusted_judgments,orig__unit_id,orig__unit_state,X_updated_at,orig_does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech_gold,does_this_tweet_contain_hate_speech_gold_reason,does_this_tweet_contain_hate_speechconfidence,tweet_id,tweet_text
0,1,853718217,True,golden,86,NaN,The tweet uses offensive language but not hate...,0.6013,NaN,True,...,0.0,615561535.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,1.666196e+09,Warning: penny boards will make you a faggot
1,2,853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,NaN,True,...,0.0,615561723.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.295121e+08,Fuck dykes
2,3,853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,NaN,True,...,0.0,615562039.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...


In [ ]:
data2['does_this_tweet_contain_hate_speech'].value_counts()

The tweet is not offensive                               7274
The tweet uses offensive language but not hate speech    4836
The tweet contains hate speech                           2399
Name: does_this_tweet_contain_hate_speech, dtype: int64

In [ ]:
data2['intent'] = data2['does_this_tweet_contain_hate_speech'].map({'The tweet is not offensive':'Not offensive','The tweet uses offensive language but not hate speech':'Offensive','The tweet contains hate speech':'Hate'})

In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data2['sentence'] = data2['tweet_text'].apply(data_cleaner)

In [ ]:
def fix_text(text):
    text = text.replace('rt', '')
    # text = text.replace('   ', '')
    return text.strip()
data2["sentence"] =  data2["sentence"].apply(fix_text)

In [ ]:
filter = data2["sentence"] != ""
data2 = data2[filter]

In [ ]:
data2 = data2[['sentence','intent']]
data2.head()

,sentence,intent
0,warning penny boards make faggot,Offensive
1,fuck dyk,Hate
2,least dont look like jefree starr faggot,Hate
3,fag jackie jealous neeee,Hate
4,heard bitch way back th texas wtf u talking bi...,Offensive


In [ ]:
data2['intent'].value_counts()

Not offensive    7270
Offensive        4836
Hate             2399
Name: intent, dtype: int64

In [ ]:
data2.to_csv(r'Dataset_2.txt', header=True, index=True, sep=',', mode='a')

Tokenizer

In [ ]:
data2['input_ids']=data2['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data2.head(3)

,sentence,intent,input_ids
0,warning penny boards make faggot,Offensive,"[101, 5432, 10647, 7923, 2191, 6904, 13871, 41..."
1,fuck dyk,Hate,"[101, 6616, 1040, 15922, 102]"
2,least dont look like jefree starr faggot,Hate,"[101, 2560, 2123, 2102, 2298, 2066, 15333, 233..."


In [ ]:
data2['intent'] = data2['intent'].map({'Hate':0,'Offensive':1,'Not offensive':3})
data2.intent.value_counts()

3    7270
1    4836
0    2399
Name: intent, dtype: int64

In [ ]:
tokenize_data2 = data2[['input_ids','intent']]
tokenize_data2.head(2)

,input_ids,intent
0,"[101, 5432, 10647, 7923, 2191, 6904, 13871, 41...",1
1,"[101, 6616, 1040, 15922, 102]",0


In [ ]:
tokenize_data2.to_csv(r'dataset2_tokenized.txt', header=True, index=True, sep=',', mode='a')

### date3

In [ ]:
data3 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_3.csv")
data3.head(3)

,text,label,gender,template
0,hate chicks Full of sound and fury Signifying ...,offensive,female,verb_adj
1,murder guy I wish I could write as mysterious ...,offensive,male,verb_adj
2,love dads i have many hobbies including juggli...,non_offensive,male,verb_adj


In [ ]:
data3.label.value_counts()

offensive        38254
non_offensive    38243
Name: label, dtype: int64

In [ ]:
data3['intent'] = data3['label'].map({'offensive':'Offensive','non_offensive':'Not offensive'})

In [ ]:
data3.intent.value_counts()

Offensive        38254
Not offensive    38243
Name: intent, dtype: int64

In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data3['sentence'] = data3['text'].apply(data_cleaner)

In [ ]:
data3.sentence.tolist()

['hate chicks full sound fury signifying noth',
 'murder guy wish could write mysterious cat',
 'love dads many hobbies including juggling slacklining baking things swimming video gam',
 'destroy chicks scanner see head heart see us clearly dark',
 'kill gals give liberty know utter argue freely according conscience liberti',
 'love brothers hope sees clearly longer see see murk',
 'hate actors possessions moment tim',
 'murder actor god everything permit',
 'destroy spokesman nothing either good bad thinking mak',
 'like mr music food love play',
 'like son come back door bathroom window like infamous beatles song',
 'hate gals scanner see head heart see us clearly dark',
 'destroy see seem dream within dream',
 'hate dudes cynic man knows price everything value noth',
 'love granddaughter far far better thing ever don',
 'murder princes see seem dream within dream',
 'hate music food love play',
 'hate mom music food love play',
 'destroy brother welcome planet earth many trees surfa

In [ ]:
data3.head()

,text,label,gender,template,intent,sentence
0,hate chicks Full of sound and fury Signifying ...,offensive,female,verb_adj,Offensive,hate chicks full sound fury signifying noth
1,murder guy I wish I could write as mysterious ...,offensive,male,verb_adj,Offensive,murder guy wish could write mysterious cat
2,love dads i have many hobbies including juggli...,non_offensive,male,verb_adj,Not offensive,love dads many hobbies including juggling slac...
3,destroy chicks What does a scanner see? Into t...,offensive,female,verb_adj,Offensive,destroy chicks scanner see head heart see us c...
4,kill gals Give me the liberty to know to utter...,offensive,female,verb_adj,Offensive,kill gals give liberty know utter argue freely...


In [ ]:
data3.dropna(inplace = True)
data3.isna().sum()

text        0
label       0
gender      0
template    0
intent      0
sentence    0
dtype: int64

In [ ]:
data3 = data3[['sentence','intent']]
data3.head(3)

,sentence,intent
0,hate chicks full sound fury signifying noth,Offensive
1,murder guy wish could write mysterious cat,Offensive
2,love dads many hobbies including juggling slac...,Not offensive


In [ ]:
data3.to_csv(r'Dataset_3.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data3['input_ids']=data3['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data3.head(3)

,sentence,intent,input_ids
0,hate chicks full sound fury signifying noth,Offensive,"[101, 5223, 20649, 2440, 2614, 8111, 3696, 117..."
1,murder guy wish could write mysterious cat,Offensive,"[101, 4028, 3124, 4299, 2071, 4339, 8075, 4937..."
2,love dads many hobbies including juggling slac...,Not offensive,"[101, 2293, 3611, 2015, 2116, 7570, 27982, 216..."


In [ ]:
data3['intent'] = data3['intent'].map({'Offensive':1,'Not offensive':3})
data3.intent.value_counts()

1    38254
3    38243
Name: intent, dtype: int64

In [ ]:
tokenize_data3 = data3[['input_ids','intent']]
tokenize_data3.head(2)

,input_ids,intent
0,"[101, 5223, 20649, 2440, 2614, 8111, 3696, 117...",1
1,"[101, 4028, 3124, 4299, 2071, 4339, 8075, 4937...",1


In [ ]:
tokenize_data3.to_csv(r'dataset3_tokenized.txt', header=True, index=True, sep=',', mode='a')

### data4

In [ ]:
import json
data4 = pd.read_json('/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_4.json', orient='index')
data4.head(3)

,post_id,annotators,rationales,post_tokens
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ..."
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem..."
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]"


In [ ]:
lst = []
for i in data4['post_tokens'].tolist():
  # print(i)
  lst.append(' '.join(i))

In [ ]:
data4['text'] = lst
data4.head(3)

,post_id,annotators,rationales,post_tokens,text
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ...",i dont think im getting my baby them white 9 h...
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem...",we cannot continue calling ourselves feminists...
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]",nawt yall niggers ignoring me


In [ ]:
label = []
for i in data4['annotators']:
  ans = list(i[0].values())
  label.append(ans[0])

In [ ]:
data4['label'] = label

In [ ]:
data4 = data4[['text','label']]
data4

,text,label
1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,normal
1179063826874032128_twitter,we cannot continue calling ourselves feminists...,normal
1178793830532956161_twitter,nawt yall niggers ignoring me,normal
1179088797964763136_twitter,<user> i am bit confused coz chinese ppl can n...,hatespeech
1179085312976445440_twitter,this bitch in whataburger eating a burger with...,hatespeech
...,...,...
9989999_gab,if ur still on twitter tell carlton i said his...,offensive
9990225_gab,when i first got on here and said i hate trump...,offensive
9991681_gab,was macht der moslem wenn der zion gegen seine...,offensive
9992513_gab,it is awful look at world demographics asians ...,hatespeech


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data4['sentence'] = data4['text'].apply(data_cleaner)

In [ ]:
data4.head(3)

,text,label,sentence
1179055004553900032_twitter,i dont think im getting my baby them white 9 h...,normal,dont think im getting baby white two white j n...
1179063826874032128_twitter,we cannot continue calling ourselves feminists...,normal,cannot continue calling feminists rights womxn...
1178793830532956161_twitter,nawt yall niggers ignoring me,normal,nawt yall niggers ignor


In [ ]:
data4.label.value_counts()

normal        8209
hatespeech    5991
offensive     5948
Name: label, dtype: int64

In [ ]:
data4['intent'] = data4['label'].map({'normal':'Normal','hatespeech':'Hate','offensive':'Offensive'})

In [ ]:
data4 = data4[['sentence','intent']]
data4.head(2)

,sentence,intent
1179055004553900032_twitter,dont think im getting baby white two white j n...,Normal
1179063826874032128_twitter,cannot continue calling feminists rights womxn...,Normal


In [ ]:
data4 = data4.reset_index()

In [ ]:
data4 = data4[['sentence','intent']]
data4.head(3)

,sentence,intent
0,dont think im getting baby white two white j n...,Normal
1,cannot continue calling feminists rights womxn...,Normal
2,nawt yall niggers ignor,Normal


In [ ]:
data4.intent.value_counts()

Normal       8209
Hate         5991
Offensive    5948
Name: intent, dtype: int64

In [ ]:
data4.to_csv(r'Dataset_4.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data4['input_ids']=data4['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data4.head(3)

,sentence,intent,input_ids
0,dont think im getting baby white two white j n...,Normal,"[101, 2123, 2102, 2228, 10047, 2893, 3336, 231..."
1,cannot continue calling feminists rights womxn...,Normal,"[101, 3685, 3613, 4214, 10469, 2015, 2916, 241..."
2,nawt yall niggers ignor,Normal,"[101, 6583, 26677, 8038, 3363, 9152, 13327, 20..."


In [ ]:
data4['intent'] = data4['intent'].map({'Offensive':1,'Hate':0,'Normal':4})
data4.intent.value_counts()

4    8209
0    5991
1    5948
Name: intent, dtype: int64

In [ ]:
tokenize_data4 = data4[['input_ids','intent']]
tokenize_data4.head(2)

,input_ids,intent
0,"[101, 2123, 2102, 2228, 10047, 2893, 3336, 231...",4
1,"[101, 3685, 3613, 4214, 10469, 2015, 2916, 241...",4


In [ ]:
tokenize_data4.to_csv(r'dataset4_tokenized.txt', header=True, index=True, sep=',', mode='a')

###data5

In [ ]:
data5 = pd.DataFrame()
folderpath = "/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_5/all_files"
inputlst = [os.listdir(folderpath)]
filenamelist = []
for filename in os.listdir(folderpath):
  words = open(os.path.join("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_5/all_files",filename), "r").read()
  dic_new = {'text':words.strip()}
  data5 =  data5.append(dic_new, ignore_index=True)

In [ ]:
data5.head()

,text
0,BLOOD PASSOVER COMPLETE ENGLISH TEXT The whole...
1,This is something we need to point out .
2,Most look the same or have the same features .
3,That is all .
4,"A Kosher slaughter , blessed by every Rabbi un..."


In [ ]:
label = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_5/annotations_metadata.csv")
label.head(3)

,file_id,user_id,subforum_id,num_contexts,label
0,12834217_1,572066,1346,0,noHate
1,12834217_2,572066,1346,0,noHate
2,12834217_3,572066,1346,0,noHate


In [ ]:
result = pd.concat([data5,label['label']], axis=1)
result.head()

,text,label
0,BLOOD PASSOVER COMPLETE ENGLISH TEXT The whole...,noHate
1,This is something we need to point out .,noHate
2,Most look the same or have the same features .,noHate
3,That is all .,hate
4,"A Kosher slaughter , blessed by every Rabbi un...",noHate


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
result['text'] = result['text'].apply(data_cleaner)

In [ ]:
result.text.tolist()

['blood passover complete english text whole english text bloody passover online url',
 'something need point',
 'look featur',
 '',
 'kosher slaughter blessed every rabbi sun',
 'speak article started thread',
 'say fitting punishment dealing developmentally challenged children rest lif',
 'leaving week actual election hope right first heard suggest',
 'get picture old boss compar',
 'reality screwing punishment nothing deters rehabilit',
 'many groups ao wonder much toe stepping going occur hope none people get hurt',
 'video center kold news click link video vekol valley looks like south interstate miles west hwy intersect',
 'pornography prostitution outlawed jewish filth nazi org uk',
 'let hope nearly white gold medal us women soccer team gets spirit',
 'death penalty anyone involved human trafficking forced prostitution anyone involving children',
 'think girls go clubs meet guys cause many guys makes easier meet one atleast thats told im danc',
 'negat',
 'eyes see clear',
 'ne

In [ ]:
filter = result["text"] != ""
data5 = result[filter]

In [ ]:
data5.dropna(inplace = True)
data5.isna().sum()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


text     0
label    0
dtype: int64

In [ ]:
data5 = data5.rename(columns={'text':'sentence','label': 'intent'})
data5.head(3)

,sentence,intent
0,blood passover complete english text whole eng...,noHate
1,something need point,noHate
2,look featur,noHate


In [ ]:
label_counts = data5.intent.value_counts()
label_counts

noHate      9378
hate        1176
relation     166
idk/skip      73
Name: intent, dtype: int64

In [ ]:
data5 = data5[data5['intent'].isin(label_counts[label_counts > 200].index)]
data5.head()

,sentence,intent
0,blood passover complete english text whole eng...,noHate
1,something need point,noHate
2,look featur,noHate
4,kosher slaughter blessed every rabbi sun,noHate
5,speak article started thread,noHate


In [ ]:
data5.intent.value_counts()

noHate    9378
hate      1176
Name: intent, dtype: int64

In [ ]:
data5['intent'] = data5['intent'].map({'noHate':'No Hate','hate':'Hate'})

In [ ]:
data5.to_csv(r'Dataset_5.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data5['input_ids']=data5['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data5.head(3)

,sentence,intent,input_ids
0,blood passover complete english text whole eng...,No Hate,"[101, 2668, 3413, 7840, 3143, 2394, 3793, 2878..."
1,something need point,No Hate,"[101, 2242, 2342, 2391, 102]"
2,look featur,No Hate,"[101, 2298, 8658, 3126, 102]"


In [ ]:
data5['intent'] = data5['intent'].map({'Hate':0,'No Hate':5})
data5.intent.value_counts()

5    9378
0    1176
Name: intent, dtype: int64

In [ ]:
tokenize_data5 = data5[['input_ids','intent']]
tokenize_data5.head(2)

,input_ids,intent
0,"[101, 2668, 3413, 7840, 3143, 2394, 3793, 2878...",5
1,"[101, 2242, 2342, 2391, 102]",5


In [ ]:
tokenize_data5.to_csv(r'dataset5_tokenized.txt', header=True, index=True, sep=',', mode='a')

### data7

In [ ]:
data7 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_7.csv")
data7 = data7[['text','label']]
data7.head()

,text,label
0,It should be legal for a husband to rape his wife,hate
1,dalits are lowlife scum,hate
2,dalits are lowlives,hate
3,It was a better world when women dare not ques...,hate
4,I don't work this hard so that those immigrant...,hate


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data7['sentence'] = data7['text'].apply(data_cleaner)

In [ ]:
data7.sentence.tolist()

['legal husband rape wif',
 'dalits lowlife scum',
 'dalits lowl',
 'better world women dare question men',
 'work hard immigrants take benefit',
 'work hard migrants take benefit',
 'work hard foreigners take benefit',
 'work hard hispanics take benefit',
 'work hard chavs take benefit',
 'work hard pakis take benefit',
 'work hard muslims take benefit',
 'foreigners burden societi',
 'immigrants burden societi',
 'women burden societi',
 'gay people burden societi',
 'gay people burden societi',
 'foreigners smell weird wash',
 'french people smell weird wash',
 'white people smell weird wash',
 'asian people smell weird wash',
 'black people smell weird wash',
 'africans smell weird wash',
 'african people smell weird wash',
 'whites smell weird wash',
 'dikes smell weird wash',
 'welsh people smell weird wash',
 'dont hate hindus',
 'dont hate muslim',
 'dont hatefucking islam',
 'dont hate fucking muslim',
 'world full women unfair',
 'government incompet',
 'short people somehow 

In [ ]:
data7.head()

,text,label,sentence
0,It should be legal for a husband to rape his wife,hate,legal husband rape wif
1,dalits are lowlife scum,hate,dalits lowlife scum
2,dalits are lowlives,hate,dalits lowl
3,It was a better world when women dare not ques...,hate,better world women dare question men
4,I don't work this hard so that those immigrant...,hate,work hard immigrants take benefit


In [ ]:
data7.dropna(inplace = True)
data7.isna().sum()

text        0
label       0
sentence    0
dtype: int64

In [ ]:
data7 = data7.rename(columns={'label': 'intent'})
data7.head(3)

,text,intent,sentence
0,It should be legal for a husband to rape his wife,hate,legal husband rape wif
1,dalits are lowlife scum,hate,dalits lowlife scum
2,dalits are lowlives,hate,dalits lowl


In [ ]:
data7 = data7[['sentence','intent']]
data7.head(2)

,sentence,intent
0,legal husband rape wif,hate
1,dalits lowlife scum,hate


In [ ]:
data7.intent.value_counts()

hate       22124
nothate    18499
Name: intent, dtype: int64

In [ ]:
data7['intent'] = data7['intent'].map({'hate':'Hate','nothate':'No Hate'})

In [ ]:
data7.to_csv(r'Dataset_7.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data7['input_ids']=data7['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data7.head(3)

,sentence,intent,input_ids
0,legal husband rape wif,Hate,"[101, 3423, 3129, 9040, 15536, 2546, 102]"
1,dalits lowlife scum,Hate,"[101, 29095, 3215, 2659, 15509, 8040, 2819, 102]"
2,dalits lowl,Hate,"[101, 29095, 3215, 2659, 2140, 102]"


In [ ]:
data7['intent'] = data7['intent'].map({'Hate':0,'No Hate':5})
data7.intent.value_counts()

0    22124
5    18499
Name: intent, dtype: int64

In [ ]:
tokenize_data7 = data7[['input_ids','intent']]
tokenize_data7.head(2)

,input_ids,intent
0,"[101, 3423, 3129, 9040, 15536, 2546, 102]",0
1,"[101, 29095, 3215, 2659, 15509, 8040, 2819, 102]",0


In [ ]:
tokenize_data7.to_csv(r'dataset7_tokenized.txt', header=True, index=True, sep=',', mode='a')

### data8

In [ ]:
data8 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/dataset_8.csv")
data8

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [ ]:
data8 = data8[['tweet','label']]
data8.head()

,tweet,label
0,@user when a father is dysfunctional and is s...,0
1,@user @user thanks for #lyft credit i can't us...,0
2,bihday your majesty,0
3,#model i love u take with u all the time in ...,0
4,factsguide: society now #motivation,0


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data8['sentence'] = data8['tweet'].apply(data_cleaner)

In [ ]:
filter = data8["sentence"] != ""
data8 = data8[filter]

In [ ]:
data8.head()

,tweet,label,sentence
0,@user when a father is dysfunctional and is s...,0,father dysfunctional selfish drags kids dysfunct
1,@user @user thanks for #lyft credit i can't us...,0,thanks credit use cause offer wheelchair vans pdx
2,bihday your majesty,0,bihday majesti
3,#model i love u take with u all the time in ...,0,love u take u time ur
4,factsguide: society now #motivation,0,factsguide societi


In [ ]:
data8.dropna(inplace = True)
data8.isna().sum()

tweet       0
label       0
sentence    0
dtype: int64

In [ ]:
data8 = data8.rename(columns={'label': 'intent'})
data8.head()

,tweet,intent,sentence
0,@user when a father is dysfunctional and is s...,0,father dysfunctional selfish drags kids dysfunct
1,@user @user thanks for #lyft credit i can't us...,0,thanks credit use cause offer wheelchair vans pdx
2,bihday your majesty,0,bihday majesti
3,#model i love u take with u all the time in ...,0,love u take u time ur
4,factsguide: society now #motivation,0,factsguide societi


In [ ]:
data8.intent.value_counts()

0    29211
1     2231
Name: intent, dtype: int64

In [ ]:
data8['intent'] = data8['intent'].map({0:'No Hate',1:'Hate'})

In [ ]:
data8 = data8[['sentence','intent']]

In [ ]:
data8.head(2)

,sentence,intent
0,father dysfunctional selfish drags kids dysfunct,No Hate
1,thanks credit use cause offer wheelchair vans pdx,No Hate


In [ ]:
data8.to_csv(r'Dataset_8.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data8['input_ids']=data8['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data8.head(3)

,sentence,intent,input_ids
0,father dysfunctional selfish drags kids dysfunct,No Hate,"[101, 2269, 28466, 2389, 14337, 8011, 2015, 42..."
1,thanks credit use cause offer wheelchair vans pdx,No Hate,"[101, 4283, 4923, 2224, 3426, 3749, 13204, 219..."
2,bihday majesti,No Hate,"[101, 12170, 14945, 4710, 16686, 4355, 2072, 102]"


In [ ]:
data8['intent'] = data8['intent'].map({'Hate':0,'No Hate':5})
data8.intent.value_counts()

5    29211
0     2231
Name: intent, dtype: int64

In [ ]:
tokenize_data8 = data8[['input_ids','intent']]
tokenize_data8.head(2)

,input_ids,intent
0,"[101, 2269, 28466, 2389, 14337, 8011, 2015, 42...",5
1,"[101, 4283, 4923, 2224, 3426, 3749, 13204, 219...",5


In [ ]:
tokenize_data8.to_csv(r'dataset8_tokenized.txt', header=True, index=True, sep=',', mode='a')

###data10

In [ ]:
data10_v1 = pd.read_csv ("/content/drive/MyDrive/Final Data MSc Project/dataset_10/SWAD v1.tsv", sep = '\t')
data10_v1 = data10_v1[['Tweets','Label']]
data10_v1.head()

,Tweets,Label
0,"@USER Someone should'veTaken"" this piece of <b...",No
1,@USER @USER Oh noes! Tough <b>shit</b>,Yes
2,@USER @USER @USER @USER LOL!!! Throwing the <b...,Yes
3,@USER @USER Da <b>fuck</b> is going on people?...,Yes
4,@USER <b>fuck</b> off,Yes


In [ ]:
data10_v2 = pd.read_csv ("/content/drive/MyDrive/Final Data MSc Project/dataset_10/SWAD v2.tsv", sep = '\t')
# data6_v2 = data6_v2[["@USER Someone should'veTaken" this piece of <b>shit</b> to a volcano. 😂",'Label']]
data10_v2.head()

,1,"@USER Someone should'veTaken"" this piece of <b>shit</b> to a volcano. 😂""",No
0,2,@USER @USER Oh noes! Tough <b>shit</b>,Yes
1,3,@USER @USER @USER @USER LOL!!! Throwing the <b...,Yes
2,4,@USER @USER Da <b>fuck</b> is going on people?...,Yes
3,5,@USER <b>fuck</b> off,Yes
4,6,@USER @USER The prison system is so <b>fucked<...,No


In [ ]:
data10_v2 = data10_v2.rename(columns={data10_v2.columns[1]: 'Tweets'})
data10_v2 = data10_v2.rename(columns={data10_v2.columns[2]: 'Label'})

In [ ]:
data10_v2 = data10_v2[['Tweets','Label']]
data10_v2.head()

,Tweets,Label
0,@USER @USER Oh noes! Tough <b>shit</b>,Yes
1,@USER @USER @USER @USER LOL!!! Throwing the <b...,Yes
2,@USER @USER Da <b>fuck</b> is going on people?...,Yes
3,@USER <b>fuck</b> off,Yes
4,@USER @USER The prison system is so <b>fucked<...,No


In [ ]:
data_10 = pd.concat([data10_v1,data10_v2], ignore_index=True)
data_10.index

RangeIndex(start=0, stop=4088, step=1)

In [ ]:
data_10.head()

,Tweets,Label
0,"@USER Someone should'veTaken"" this piece of <b...",No
1,@USER @USER Oh noes! Tough <b>shit</b>,Yes
2,@USER @USER @USER @USER LOL!!! Throwing the <b...,Yes
3,@USER @USER Da <b>fuck</b> is going on people?...,Yes
4,@USER <b>fuck</b> off,Yes


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data_10['sentence'] = data_10['Tweets'].apply(data_cleaner)

In [ ]:
data_10.dropna(inplace = True)
data_10.isna().sum()

Tweets      0
Label       0
sentence    0
dtype: int64

In [ ]:
data10 = data_10.rename(columns={'Label': 'intent'})
data10.head()

,Tweets,intent,sentence
0,"@USER Someone should'veTaken"" this piece of <b...",No,someone vetaken piece shit volcano
1,@USER @USER Oh noes! Tough <b>shit</b>,Yes,oh noes tough shit
2,@USER @USER @USER @USER LOL!!! Throwing the <b...,Yes,lol throwing bullshit flag nonsense url
3,@USER @USER Da <b>fuck</b> is going on people?...,Yes,da fuck going people men room women room pick ...
4,@USER <b>fuck</b> off,Yes,fuck


In [ ]:
data10 = data10[['sentence','intent']]
data10.head(3)

,sentence,intent
0,someone vetaken piece shit volcano,No
1,oh noes tough shit,Yes
2,lol throwing bullshit flag nonsense url,Yes


In [ ]:
data10.intent.value_counts()

No     2626
Yes    1462
Name: intent, dtype: int64

In [ ]:
data10['intent'] = data10['intent'].map({'No':'No Hate','Yes':'Hate'})

In [ ]:
data10.to_csv(r'Dataset_10.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data10['input_ids']=data10['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data10.head(3)

,sentence,intent,input_ids
0,someone vetaken piece shit volcano,No Hate,"[101, 2619, 29525, 13808, 2078, 3538, 4485, 12..."
1,oh noes tough shit,Hate,"[101, 2821, 2053, 2229, 7823, 4485, 102]"
2,lol throwing bullshit flag nonsense url,Hate,"[101, 8840, 2140, 6886, 14636, 5210, 14652, 24..."


In [ ]:
data10['intent'] = data10['intent'].map({'Hate':0,'No Hate':5})
data10.intent.value_counts()

5    2626
0    1462
Name: intent, dtype: int64

In [ ]:
tokenize_data10 = data10[['input_ids','intent']]
tokenize_data10.head(2)

,input_ids,intent
0,"[101, 2619, 29525, 13808, 2078, 3538, 4485, 12...",5
1,"[101, 2821, 2053, 2229, 7823, 4485, 102]",0


In [ ]:
tokenize_data10.to_csv(r'dataset10_tokenized.txt', header=True, index=True, sep=',', mode='a')

### data6

In [ ]:
data6 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/classified_tweets.csv")
data6.head(4)

,text,suspicious,cyberbullying,hate,suicidal
0,Uhmm like 6th grade on a corner of a street....,0,0,0,0
1,a) JTP is a douchebag b) Stewart kicks ass!,1,0,0,0
2,ditto bitch!,1,0,0,0
3,damn I have to drive my dad to the airport tha...,0,0,0,0


In [ ]:
data6.cyberbullying.value_counts()

0    17256
2     1733
1      945
Name: cyberbullying, dtype: int64

In [ ]:
data6 = data6[['text','cyberbullying']]
data6.head(3)

,text,cyberbullying
0,Uhmm like 6th grade on a corner of a street....,0
1,a) JTP is a douchebag b) Stewart kicks ass!,0
2,ditto bitch!,0


In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data6['sentence'] = data6['text'].apply(data_cleaner)

In [ ]:
def fix_text(text):
    text = text.replace('rt', '')
    text = text.replace('hahahah', '')
    text = text.replace('ahaha', '')
    text = text.replace('vvvvvvvummmmmmmmm', '')
    text = text.replace('camp', '')
    text = text.replace('sureee', '')
    text = text.replace('yea', '')
    text = text.replace('guesss', '')
    text = text.replace('blow', '')
    text = text.replace('lie', '')
    text = text.replace('usa', '')
    text = text.replace('nope', '')
    text = text.replace('nahhh', '')
    text = text.replace('tri', '')
    text = text.replace('yes', '')
    text = text.replace('yeahh', '')
    text = text.replace('got', '')
    text = text.replace('star', '')
    text = text.replace('hmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm', '')

    return text.strip()
data6["sentence"] =  data6["sentence"].apply(fix_text)

In [ ]:
filter = data6["sentence"] != ""
data6 = data6[filter]

In [ ]:
data6.head(2)

,text,cyberbullying,sentence
0,Uhmm like 6th grade on a corner of a street....,0,uhmm like th grade corner street corner lol jk...
1,a) JTP is a douchebag b) Stewart kicks ass!,0,jtp douchebag b stewa kicks ass


In [ ]:
data6['intent'] = data6['cyberbullying'].map({0:'Neither',1:'Racism',2:'Sexism'})
data6.head(2)

,text,cyberbullying,sentence,intent
0,Uhmm like 6th grade on a corner of a street....,0,uhmm like th grade corner street corner lol jk...,Neither
1,a) JTP is a douchebag b) Stewart kicks ass!,0,jtp douchebag b stewa kicks ass,Neither


In [ ]:
data6 = data6[['sentence','intent']]
data6.head(2)

,sentence,intent
0,uhmm like th grade corner street corner lol jk...,Neither
1,jtp douchebag b stewa kicks ass,Neither


In [ ]:
data6.to_csv(r'Dataset_6.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data6['input_ids']=data6['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data6.head(3)

,sentence,intent,input_ids
0,uhmm like th grade corner street corner lol jk...,Neither,"[101, 7910, 7382, 2066, 16215, 3694, 3420, 239..."
1,jtp douchebag b stewa kicks ass,Neither,"[101, 1046, 25856, 2079, 19140, 16078, 1038, 2..."
2,ditto bitch,Neither,"[101, 4487, 9284, 7743, 102]"


In [ ]:
data6['intent'] = data6['intent'].map({'Neither':2,'Racism':6,'Sexism':7})
data6.intent.value_counts()

2    16909
7     1716
6      945
Name: intent, dtype: int64

In [ ]:
tokenize_data6 = data6[['input_ids','intent']]
tokenize_data6.head(2)

,input_ids,intent
0,"[101, 7910, 7382, 2066, 16215, 3694, 3420, 239...",2
1,"[101, 1046, 25856, 2079, 19140, 16078, 1038, 2...",2


In [ ]:
tokenize_data6.to_csv(r'dataset6_tokenized.txt', header=True, index=True, sep=',', mode='a')

###data9

In [ ]:
data9 = pd.read_csv("/content/drive/MyDrive/Final Data MSc Project/DataSets/cyberbullying_tweets.csv")
data9.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data9.cyberbullying_type.value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [ ]:
data9 = data9[(data9['cyberbullying_type']=='not_cyberbullying') | (data9['cyberbullying_type']=='other_cyberbullying')]
data9.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
data9.cyberbullying_type.value_counts()

not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [ ]:
!pip install demoji --q

     |████████████████████████████████| 42 kB 899 kB/s 


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import demoji
import string
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.update(['rt', 'mkr', 'didn', 'bc', 'n', 'm', 
                  'im', 'll', 'y', 've', 'u', 'ur', 'don', 
                  'p', 't', 's', 'aren', 'kp', 'o', 'kat', 
                  'de', 're', 'amp', 'will', 'wa', 'e', 'like'])
stemmer = SnowballStemmer('english')
def clean_text(text):
    
    # Remove Hashtag, Mention, URLs
    pattern = re.compile(r"(#[A-Za-z0-9]+|@[A-Za-z0-9]+|https?://\S+|www\.\S+|\S+\.[a-z]+|RT @)")
    text = pattern.sub('', text)
    text = " ".join(text.split())
    
    # Make all text lowercase
    text = text.lower()
    
    # Stemming
    text = " ".join([stemmer.stem(word) for word in text.split()])
    
    # Removing Punctuations
    remove_punc = re.compile(r"[%s]" % re.escape(string.punctuation))
    text = remove_punc.sub('', text)
    
    # Removing stopwords
    text = " ".join([word for word in str(text).split() if word not in STOPWORDS])
    
    # Taking care of emojis
    emoji = demoji.findall(text)
    for emot in emoji:
        text = re.sub(r"(%s)" % (emot), "_".join(emoji[emot].split()), text)

    return text

In [ ]:
data9['sentence'] = data9['tweet_text'].apply(lambda text: clean_text(text)) 

In [ ]:
def data_cleaner(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', ' ', tweet)   # remove urls
    tweet = re.sub(r'<.*?>',' ', tweet)      # remove html tags
    tweet = re.sub(r'\d+',' ', tweet)        # remove digits
    tweet = re.sub(r'#\w+',' ', tweet)       # remove hashtags
    tweet = re.sub(r'@\w+',' ', tweet)       # remove mentions
    tweet = re.sub(r'[^\w\s]',' ', tweet)    # remove punctuation
    tweet = re.sub('[^A-Za-z0-9 ]+', '', tweet) # remove characters that are not a letters or numbers
    tweet = " ".join([word for word in tweet.split() if not word in stop_words])   # remove stop words
    tweet = stemmer.stem(tweet)
    tweet = tweet.strip()
    return tweet
stop_words = stopwords.words('english')
stemmer = SnowballStemmer("english")

#X.apply(data_cleaner)
data9['sentence'] = data9['sentence'].apply(data_cleaner)

In [ ]:
filter = data9["sentence"] != ""
data9 = data9[filter]

In [ ]:
data9.head(2)

,tweet_text,cyberbullying_type,sentence
0,"In other words #katandandre, your food was cra...",not_cyberbullying,word food crapilici
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,whi whit


In [ ]:
data9['intent'] = data9['cyberbullying_type'].map({'not_cyberbullying':'Not cyberbullying','other_cyberbullying':'Cyberbullying'})
data9.head(3)

,tweet_text,cyberbullying_type,sentence,intent
0,"In other words #katandandre, your food was cra...",not_cyberbullying,word food crapilici,Not cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,whi whit,Not cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,classi whore red velvet cupcak,Not cyberbullying


In [ ]:
data9 = data9[['sentence','intent']]
data9.head(3)

,sentence,intent
0,word food crapilici,Not cyberbullying
1,whi whit,Not cyberbullying
2,classi whore red velvet cupcak,Not cyberbullying


In [ ]:
data9.to_csv(r'Dataset_9.txt', header=True, index=True, sep=',', mode='a')

In [ ]:
data9['input_ids']=data9['sentence'].apply(lambda x: tokenizer(x,  padding='longest')['input_ids'])
data9.head(3)

Token indices sequence length is longer than the specified maximum sequence length for this model (580 > 512). Running this sequence through the model will result in indexing errors


,sentence,intent,input_ids
0,word food crapilici,Not cyberbullying,"[101, 2773, 2833, 10231, 18622, 6895, 102]"
1,whi whit,Not cyberbullying,"[101, 1059, 4048, 1059, 16584, 102]"
2,classi whore red velvet cupcak,Not cyberbullying,"[101, 2465, 2072, 17219, 2417, 10966, 2452, 35..."


In [ ]:
data9['intent'] = data9['intent'].map({'Not cyberbullying':8,'Cyberbullying':9})
data9.intent.value_counts()

8    7790
9    7601
Name: intent, dtype: int64

In [ ]:
tokenize_data9 = data9[['input_ids','intent']]
tokenize_data9.head(3)

,input_ids,intent
0,"[101, 2773, 2833, 10231, 18622, 6895, 102]",8
1,"[101, 1059, 4048, 1059, 16584, 102]",8
2,"[101, 2465, 2072, 17219, 2417, 10966, 2452, 35...",8


In [ ]:
tokenize_data9.to_csv(r'dataset9_tokenized.txt', header=True, index=True, sep=',', mode='a')
